## 1. Run retrosynthesis planning

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from CGRtools import smiles

from Synto.mcts import Tree
from Synto.utils.config import read_planning_config

In [3]:
# planning config

planning_config = {
    "General": {
        "reaction_rules_path": '../synto_data/reaction_rules.pickle',
        "building_blocks_path": '../synto_data/building_blocks.txt',
    },
    "Tree": {
        "ucb_type": "uct",
        "c_usb": 0.1,
        "max_depth": 6,
        "max_iterations": 50,
        "max_time": 600,
        "max_tree_size": 1e6,
        "verbose": True,
        "evaluation_mode": "gcn",
        "evaluation_agg": "max",
        "backprop_type": "muzero",
        "init_new_node_value": None,
        "epsilon": 0.0
    },
    "PolicyNetwork": {
        "weights_path": '../synto_data/policy_network.ckpt',
        "priority_rules_fraction": 0.5,
        "top_rules": 50,
        "rule_prob_threshold": 0.0
    },
    "ValueNetwork": {
        "weights_path": '../synto_data/value_network.ckpt'
    },
}

In [4]:
# preprocess target molecule

target = 'CC(C)CCN1CCN(C2CSCCSC2)CC1CCO'

target = smiles(target)
target.canonicalize()
target.clean2d()
target

In [5]:
# run retrosynthesis planning with MCTS

tree = Tree(target=target, config=planning_config)
_ = list(tree)

# print short planning report
print(tree)

100%|█████████████████████████████████████████████| 50/50 [00:07<00:00,  6.64it/s]

Tree for: C1SCCSCC1N2CC(CCO)N(CCC(C)C)CC2
Number of nodes: 413
Number of visited nodes: 22
Found paths: 39
Time: 6.7 seconds


## 2. Analyze planning results

In [6]:
print(tree)

Tree for: C1SCCSCC1N2CC(CCO)N(CCC(C)C)CC2
Number of nodes: 413
Number of visited nodes: 22
Found paths: 39
Time: 6.7 seconds


### We can get a list of winning nodes (IDs of leaf nodes in found retrosynthetic paths) by:

In [7]:
tree.winning_nodes

[78,
 81,
 82,
 84,
 88,
 90,
 78,
 81,
 82,
 84,
 88,
 90,
 179,
 181,
 182,
 186,
 189,
 190,
 191,
 192,
 193,
 191,
 190,
 181,
 193,
 189,
 192,
 182,
 186,
 179,
 389,
 393,
 403,
 404,
 405,
 406,
 407,
 408,
 356]

In [8]:
success_node = tree.winning_nodes[0]

tree.path_to_node(success_node)

[retrons_to_expand: (C1SCCSCC1N2CC(CCO)N(CCC(C)C)CC2,)
 new_retrons: (C1SCCSCC1N2CC(CCO)N(CCC(C)C)CC2,),
 retrons_to_expand: (CC(C)CCN1CCNCC1CCO,)
 new_retrons: (CC(C)CCN1CCNCC1CCO, C1SCCSCC1=O),
 retrons_to_expand: ()
 new_retrons: (CC(C)CC(=O)O, OCCC1NCCNC1)]

In [9]:
tree.synthesis_path(success_node)

(<Synto.chem.reaction.Reaction at 0x7fcc7f5e02e0>,
 <Synto.chem.reaction.Reaction at 0x7fcc7f5e0350>)

In [10]:
tree.path_score(success_node)

0.09392555885844761

## 3. Retrosynthesis path visualisation

In [11]:
from CGRtools import MoleculeContainer
MoleculeContainer.depict_settings(aam=False)

In [12]:
from IPython.display import SVG, display
from Synto.interfaces.visualisation import path_graph

In [13]:
for node_id in tree.winning_nodes:
    print(f'Found path')
    display(SVG(path_graph(tree, node_id)))
    

Found path


Found path


Found path


Found path


Found path


...

Found path


Found path


Found path


Found path


Found path


In [14]:
from cairosvg import svg2png

svg_code = path_graph(tree, node_id)

svg2png(bytestring=svg_code, write_to='retropath.png')